# <center>
<div style="
    background: #f0f333ff;
    border-left: 5px solid #ecd242ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h1 style="
        text-align: center;
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 600;
    ">
    Exercise
    </h1>
</div>

##
<div style="
    background: #40f0aa;
    border-left: 5px solid #0c7230ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h2 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    First scénario
    </h2>
</div>

Vous avez 3 employés (Alice, Bob, Charlie) et 2 postes (Poste1, Poste2) à pourvoir chaque jour pendant 3 jours.<br>
Chaque jour, chaque poste doit être tenu par exactement un employé, et un employé ne peut tenir qu'un poste par jour.<br>
De plus, nous voulons que chaque employé travaille au moins une fois sur chaque poste sur les 3 jours.<br>

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    My Solution
    </h3>
</div>

In [31]:
from ortools.sat.python import cp_model

class PartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, employees, postes, days, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._employees = employees
        self._postes = postes
        self._days = days
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        print("\t  Poste1   Poste2")
        for d in self._days:
            st = f"Day {d}  →  "
            for p in self._postes:
                for e in self._employees:
                    if self.value(self._shifts[(e, d, p)]):
                        st += f"{e:7}  "
            print(st)
        print()

        if self._solution_count >= self._solution_limit:
            print(f"\nStop search after {self._solution_limit} solutions")
            self.stop_search()

    def solutionCount(self):
        return self._solution_count


# Data.
employees = ['Alice', 'Bob', 'Charlie']
postes = ['Poste1', 'Poste2']
days = range(3)

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# shifts[(e, d, p)]: employee 'e' works day 'd' on poste 'p'.
shifts = {}
for e in employees:
    for d in days:
        for p in postes:
            shifts[(e, d, p)] = model.new_bool_var(f"shift_{e}_{d}_{p}")

# Each post must be assigned to exactly one employee each day.
for d in days:
    for p in postes:
        model.add_exactly_one(shifts[(e, d, p)] for e in employees)

# Each employee works at most one post per day.
for d in days:
    for e in employees:
        model.add_at_most_one(shifts[(e, d, p)] for p in postes)

# Each employee works at least one time on each post over the 3 days.
for p in postes:
    for e in employees:
        model.add_at_least_one(shifts[(e, d, p)] for d in days)

# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

# Display the first five solutions.
solution_limit = 3
solution_printer = PartialSolutionPrinter(shifts, employees, postes, days, solution_limit)
solver.solve(model, solution_printer)

Solution 1
	  Poste1   Poste2
Day 0  →  Bob      Charlie  
Day 1  →  Charlie  Alice    
Day 2  →  Alice    Bob      

Solution 2
	  Poste1   Poste2
Day 0  →  Charlie  Alice    
Day 1  →  Bob      Charlie  
Day 2  →  Alice    Bob      

Solution 3
	  Poste1   Poste2
Day 0  →  Charlie  Bob      
Day 1  →  Alice    Charlie  
Day 2  →  Bob      Alice    


Stop search after 3 solutions


2

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Architecture from DeepSeek
    </h3>
</div>

In [49]:
from ortools.sat.python import cp_model

def simple_planning_exercise():
    # Données du problème
    employes = ['Alice', 'Bob', 'Charlie']
    jours = ['Lundi', 'Mardi', 'Mercredi']
    
    model = cp_model.CpModel()

    # 1. Créer les variables booléennes
    shifts = {}
    for employe in employes:
        for jour in jours:
            shifts[(employe, jour)] = model.NewBoolVar(f'shift_{employe}_{jour}')

    # 2. Ajouter les contraintes ici
    # TODO: Chaque jour, exactement 2 employés travaillent
    for jour in jours:
        model.Add(sum(shifts[(employe, jour)] for employe in employes) == 2)  # À compléter

    # TODO: Chaque employé travaille au maximum 2 jours
    for employe in employes:
        model.Add(sum(shifts[(employe, jour)] for jour in jours) <= 2)  # À compléter

    # 3. Résolution
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # 4. Affichage
    if status == cp_model.OPTIMAL:
        print("📅 Planning optimal trouvé:\n")
        for jour in jours:
            print(f"{jour}: ", end="")
            for employe in employes:
                if solver.Value(shifts[(employe, jour)]):
                    print(f"{employe} ", end="")
            print()
    else:
        print("Aucune solution trouvée")

    print(len(shifts))
    print(shifts)

# Lancez l'exercice
simple_planning_exercise()


📅 Planning optimal trouvé:

Lundi: Bob Charlie 
Mardi: Alice Bob 
Mercredi: Alice Charlie 
9
{('Alice', 'Lundi'): shift_Alice_Lundi(0..1), ('Alice', 'Mardi'): shift_Alice_Mardi(0..1), ('Alice', 'Mercredi'): shift_Alice_Mercredi(0..1), ('Bob', 'Lundi'): shift_Bob_Lundi(0..1), ('Bob', 'Mardi'): shift_Bob_Mardi(0..1), ('Bob', 'Mercredi'): shift_Bob_Mercredi(0..1), ('Charlie', 'Lundi'): shift_Charlie_Lundi(0..1), ('Charlie', 'Mardi'): shift_Charlie_Mardi(0..1), ('Charlie', 'Mercredi'): shift_Charlie_Mercredi(0..1)}


##
<div style="
    background: #40f0aa;
    border-left: 5px solid #0c7230ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h2 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Augmented scénario
    </h2>
</div>

**Préférences** : Alice ne veut pas travailler le second jour.

In [52]:
from ortools.sat.python import cp_model

class PartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, employees, postes, days, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._employees = employees
        self._postes = postes
        self._days = days
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        print("\t  Poste1   Poste2")
        for d in self._days:
            st = f"Day {d}  →  "
            for p in self._postes:
                for e in self._employees:
                    if self.value(self._shifts[(e, d, p)]):
                        st += f"{e:7}  "
            print(st)
        print()

        if self._solution_count >= self._solution_limit:
            print(f"\nStop search after {self._solution_limit} solutions")
            self.stop_search()

    def solutionCount(self):
        return self._solution_count


# Data.
employees = ['Alice', 'Bob', 'Charlie', 'Dan']
postes = ['Poste1', 'Poste2']
days = range(4)

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# shifts[(e, d, p)]: employee 'e' works day 'd' on poste 'p'.
shifts = {}
for e in employees:
    for d in days:
        for p in postes:
            shifts[(e, d, p)] = model.new_bool_var(f"shift_{e}_{d}_{p}")

# Each post must be assigned to exactly one employee each day.
for d in days:
    for p in postes:
        model.add_exactly_one(shifts[(e, d, p)] for e in employees)

# Each employee works at most one post per day.
for d in days:
    for e in employees:
        model.add_at_most_one(shifts[(e, d, p)] for p in postes)

# Each employee works at least one time on each post over the 3 days.
for p in postes:
    for e in employees:
        model.add_at_least_one(shifts[(e, d, p)] for d in days)

# Alice does not want to work the second and fourth days.
model.add(shifts[('Alice', 1, 'Poste1')] == 0)
model.add(shifts[('Alice', 1, 'Poste2')] == 0)
model.add(shifts[('Dan', 0, 'Poste1')] == 0)
model.add(shifts[('Dan', 0, 'Poste2')] == 0)

# Creates the solver and solve.
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

# Display the first five solutions.
solution_limit = 20
solution_printer = PartialSolutionPrinter(shifts, employees, postes, days, solution_limit)
status = solver.solve(model, solution_printer)

if status != cp_model.OPTIMAL:
    print("No optimal solution found !")

Solution 1
	  Poste1   Poste2
Day 0  →  Bob      Charlie  
Day 1  →  Charlie  Dan      
Day 2  →  Dan      Alice    
Day 3  →  Alice    Bob      

Solution 2
	  Poste1   Poste2
Day 0  →  Bob      Alice    
Day 1  →  Charlie  Dan      
Day 2  →  Dan      Charlie  
Day 3  →  Alice    Bob      

Solution 3
	  Poste1   Poste2
Day 0  →  Charlie  Alice    
Day 1  →  Bob      Dan      
Day 2  →  Dan      Charlie  
Day 3  →  Alice    Bob      

Solution 4
	  Poste1   Poste2
Day 0  →  Charlie  Alice    
Day 1  →  Bob      Dan      
Day 2  →  Dan      Bob      
Day 3  →  Alice    Charlie  

Solution 5
	  Poste1   Poste2
Day 0  →  Charlie  Alice    
Day 1  →  Bob      Dan      
Day 2  →  Alice    Bob      
Day 3  →  Dan      Charlie  

Solution 6
	  Poste1   Poste2
Day 0  →  Bob      Alice    
Day 1  →  Charlie  Dan      
Day 2  →  Alice    Bob      
Day 3  →  Dan      Charlie  

Solution 7
	  Poste1   Poste2
Day 0  →  Bob      Alice    
Day 1  →  Charlie  Dan      
Day 2  →  Alice    Charlie  
D